In [1]:
!cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
!cp /kaggle/input/backup-806/util_openbook.py .

Processing ./datasets-2.14.4-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [2]:
# installing offline dependencies
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./sentence-transformers
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126134 sha256=13f4403d3d18122d4083341a755044303a5dd395842767b43e8366199779c282
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
Processing /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl
Processing /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
Processing /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
Processing /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl


In [3]:
import numpy as np
import pandas as pd 
from datasets import load_dataset, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import os
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
from datasets import Dataset
from torch.utils.data import DataLoader
import gc
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from __future__ import annotations

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")

from dataclasses import dataclass
from typing import Optional, Union
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer,AutoModel
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
!cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

In [5]:
SIM_MODEL = '/kaggle/input/llm-fintune6/LLM_fintune5/e5-small-v2'
DEVICE = 0
MAX_LENGTH = 512
BATCH_SIZE = 8

WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)
trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", 1)
trn.head()
model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
# model = model.half()
sentence_index = read_index("/kaggle/input/llm-fintune6/LLM_fintune5/LLM_model3/parsed_e5_2.index")
sentence_index = faiss.index_cpu_to_all_gpus(sentence_index)
## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)
## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']
prompt_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()
search_score, search_index = sentence_index.search(prompt_embeddings, 10)
del prompt_embeddings
del sentence_index
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()
paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
               
                                             num_proc=2)["temp_text"]
context_e5 = []
for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx
    parsed = []
    scr_idx = scr_idx[::-1]
    for i in range(len(idx)):
        parsed.append(modified_texts[scr_idx[i]])
    parsed_text = '\n'.join(parsed)
    context_e5.append(parsed_text)
del search_score
del search_index
del paraphs_parsed_dataset
del modified_texts
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()

/tmp/ipykernel_23/3867933544.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", 1)


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

Map (num_proc=2):   0%|          | 0/2101279 [00:00<?, ? examples/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [6]:
def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]

def get_relevant_documents_parsed(df_valid):
    df_chunk_size=600
    paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def get_relevant_documents(df_valid):
    df_chunk_size=800
    
    cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def retrieval(df_valid, modified_texts):
    
    corpus_df_valid = df_valid.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words)
    vectorizer1.fit(corpus_df_valid)
    vocab_df_valid = vectorizer1.get_feature_names_out()
    vectorizer = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_valid)
    vectorizer.fit(modified_texts[:500000])
    corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000
    top_per_chunk = 15
    top_per_query = 15

    all_chunk_top_indices = []
    all_chunk_top_values = []

    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    return articles_indices, merged_top_scores


def prepare_answering_input(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=4096,
    ):
    c_plus_q   = context + ' ' + tokenizer.bos_token + ' ' + question
    c_plus_q_4 = [c_plus_q] * len(options)
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
    )
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    example_encoded = {
        "input_ids": input_ids.to(model.device.index),
        "attention_mask": attention_mask.to(model.device.index),
    }
    return example_encoded

In [7]:
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']

In [8]:
df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")

In [9]:
retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()

Map (num_proc=2):   0%|          | 0/2101279 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'doesn', 'don', 'isn', 'll', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 11222


  0%|          | 0/22 [00:00<?, ?it/s]

In [10]:
retrieved_articles = get_relevant_documents(df_valid)
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()

Map (num_proc=2):   0%|          | 0/2781652 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'doesn', 'don', 'isn', 'll', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 11222


  0%|          | 0/28 [00:00<?, ?it/s]

In [11]:
# tokenizer_dir = "/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model2/tokenizer"
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
# config = torch.load("/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model2/config.pth")
# model = AutoModelForMultipleChoice.from_config(config)

# state = torch.load(f"/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model2/pytorch_model.bin",
#                        map_location=torch.device('cpu'))
# model.load_state_dict(state)
# model = model.cuda()
# model.eval()

In [12]:
# test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
# test_df['context'] = [f"{retrieved_article[-15][2]}\n{retrieved_article[-14][2]}\n{retrieved_article[-13][2]}\n{retrieved_article[-12][2]}\n{retrieved_article[-11][2]}\n{retrieved_article[-10][2]}\n{retrieved_article[-9][2]}\n{retrieved_article[-8][2]}\n{retrieved_article[-7][2]}\n{retrieved_article[-6][2]}\n{retrieved_article[-5][2]}\n{retrieved_article[-4][2]}\n{retrieved_article[-3][2]}\n{retrieved_article[-2][2]}\n{retrieved_article[-1][2]}" for retrieved_article in retrieved_articles]

# test_df.index = list(range(len(test_df)))
# test_df['id'] = list(range(len(test_df)))
# test_df["prompt"] = test_df["context"].apply(lambda x: x[-4920:]) + " #### " +  test_df["prompt"]
# test_df['answer'] = 'A'

# # We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
# options = 'ABCDE'
# indices = list(range(5))

# option_to_index = {option: index for option, index in zip(options, indices)}
# index_to_option = {index: option for option, index in zip(options, indices)}

# def preprocess(example):
#     # The AutoModelForMultipleChoice class expects a set of question/answer pairs
#     # so we'll copy our question 5 times before tokenizing
#     first_sentence = [example['prompt']] * 5
#     second_sentence = []
#     for option in options:
#         second_sentence.append(example[option])
#     # Our tokenizer will turn our text into token IDs BERT can understand
#     tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
#     tokenized_example['label'] = option_to_index[example['answer']]
#     return tokenized_example
# @dataclass
# class DataCollatorForMultipleChoice:
#     tokenizer: PreTrainedTokenizerBase
#     padding: Union[bool, str, PaddingStrategy] = True
#     max_length: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
    
#     def __call__(self, features):
#         label_name = "label" if 'label' in features[0].keys() else 'labels'
#         labels = [feature.pop(label_name) for feature in features]
#         batch_size = len(features)
#         num_choices = len(features[0]['input_ids'])
#         flattened_features = [
#             [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
#         ]
#         flattened_features = sum(flattened_features, [])
        
#         batch = self.tokenizer.pad(
#             flattened_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors='pt',
#         )
#         batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
#         batch['labels'] = torch.tensor(labels, dtype=torch.int64)
#         return batch
# tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
# tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
# data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
# test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
# test_predictionsj = []
# for batch in test_dataloader:
#     for k in batch.keys():
#         batch[k] = batch[k].cuda()
#     with torch.no_grad():
#         outputs = model(**batch)
#     test_predictionsj.append(outputs.logits.cpu().detach())

# test_predictionsj = torch.cat(test_predictionsj)

In [13]:
# test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
# test_df['context'] = [f"{retrieved_articles_parse[-15][2]}\n{retrieved_articles_parse[-14][2]}\n{retrieved_articles_parse[-13][2]}\n{retrieved_articles_parse[-12][2]}\n{retrieved_articles_parse[-11][2]}\n{retrieved_articles_parse[-10][2]}\n{retrieved_articles_parse[-9][2]}\n{retrieved_articles_parse[-8][2]}\n{retrieved_articles_parse[-7][2]}\n{retrieved_articles_parse[-6][2]}\n{retrieved_articles_parse[-5][2]}\n{retrieved_articles_parse[-4][2]}\n{retrieved_articles_parse[-3][2]}\n{retrieved_articles_parse[-2][2]}\n{retrieved_articles_parse[-1][2]}" for retrieved_articles_parse in retrieved_articles_parsed]

# test_df.index = list(range(len(test_df)))
# test_df['id'] = list(range(len(test_df)))
# test_df["prompt"] = test_df["context"].apply(lambda x: x[-4920:]) + " #### " +  test_df["prompt"]
# test_df['answer'] = 'A'
# # We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
# options = 'ABCDE'
# indices = list(range(5))

# option_to_index = {option: index for option, index in zip(options, indices)}
# index_to_option = {index: option for option, index in zip(options, indices)}

# def preprocess(example):
#     # The AutoModelForMultipleChoice class expects a set of question/answer pairs
#     # so we'll copy our question 5 times before tokenizing
#     first_sentence = [example['prompt']] * 5
#     second_sentence = []
#     for option in options:
#         second_sentence.append(example[option])
#     # Our tokenizer will turn our text into token IDs BERT can understand
#     tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
#     tokenized_example['label'] = option_to_index[example['answer']]
#     return tokenized_example
# @dataclass
# class DataCollatorForMultipleChoice:
#     tokenizer: PreTrainedTokenizerBase
#     padding: Union[bool, str, PaddingStrategy] = True
#     max_length: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
    
#     def __call__(self, features):
#         label_name = "label" if 'label' in features[0].keys() else 'labels'
#         labels = [feature.pop(label_name) for feature in features]
#         batch_size = len(features)
#         num_choices = len(features[0]['input_ids'])
#         flattened_features = [
#             [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
#         ]
#         flattened_features = sum(flattened_features, [])
        
#         batch = self.tokenizer.pad(
#             flattened_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors='pt',
#         )
#         batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
#         batch['labels'] = torch.tensor(labels, dtype=torch.int64)
#         return batch
# tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
# tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
# data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
# test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
# test_predictionsi = []
# for batch in test_dataloader:
#     for k in batch.keys():
#         batch[k] = batch[k].cuda()
#     with torch.no_grad():
#         outputs = model(**batch)
#     test_predictionsi.append(outputs.logits.cpu().detach())

# test_predictionsi = torch.cat(test_predictionsi)

In [14]:
model_dir = "/kaggle/input/llm-fintune6/LLM_fintune5/LLM_model3"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_aff

In [15]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = [f"{retrieved_article[-15][2]}\n{retrieved_article[-14][2]}\n{retrieved_article[-13][2]}\n{retrieved_article[-12][2]}\n{retrieved_article[-11][2]}\n{retrieved_article[-10][2]}\n{retrieved_article[-9][2]}\n{retrieved_article[-8][2]}\n{retrieved_article[-7][2]}\n{retrieved_article[-6][2]}\n{retrieved_article[-5][2]}\n{retrieved_article[-4][2]}\n{retrieved_article[-3][2]}\n{retrieved_article[-2][2]}\n{retrieved_article[-1][2]}" for retrieved_article in retrieved_articles]

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-5000:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'

# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictionsk = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictionsk.append(outputs.logits.cpu().detach())

test_predictionsk = torch.cat(test_predictionsk)
del test_dataloader
del tokenized_test_dataset
del test_df
del retrieved_articles
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = [f"{retrieved_articles_parse[-15][2]}\n{retrieved_articles_parse[-14][2]}\n{retrieved_articles_parse[-13][2]}\n{retrieved_articles_parse[-12][2]}\n{retrieved_articles_parse[-11][2]}\n{retrieved_articles_parse[-10][2]}\n{retrieved_articles_parse[-9][2]}\n{retrieved_articles_parse[-8][2]}\n{retrieved_articles_parse[-7][2]}\n{retrieved_articles_parse[-6][2]}\n{retrieved_articles_parse[-5][2]}\n{retrieved_articles_parse[-4][2]}\n{retrieved_articles_parse[-3][2]}\n{retrieved_articles_parse[-2][2]}\n{retrieved_articles_parse[-1][2]}" for retrieved_articles_parse in retrieved_articles_parsed]

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-5000:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictionsl = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictionsl.append(outputs.logits.cpu().detach())

test_predictionsl = torch.cat(test_predictionsl)
del test_dataloader
del tokenized_test_dataset
del test_df
del retrieved_articles_parsed
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [17]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = context_e5

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-5000:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictions_e5 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions_e5.append(outputs.logits.cpu().detach())

test_predictions_e5 = torch.cat(test_predictions_e5)
del model
del tokenizer
del test_dataloader
del tokenized_test_dataset
del context_e5
gc.collect()
libc.malloc_trim(0)
torch.cuda.empty_cache()


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
test_predictions = test_predictionsk*0.3+test_predictionsl*0.5+test_predictions_e5*0.2

In [19]:
predictions_as_ids = np.argsort(-test_predictions, 1)

predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
# predictions_as_answer_letters[:3]

predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)